In [1]:
from PipelineTS.dataset import LoadWebSales

In [2]:
data = LoadWebSales()[['date', 'type_a']]

In [3]:
data = data.iloc[:-30, :]
valid_data = data.iloc[-30:, :]
data.tail(5)

,date,type_a
2054,2021-08-10,9261
2055,2021-08-11,8976
2056,2021-08-12,8850
2057,2021-08-13,8731
2058,2021-08-14,7862


# PipelineTS

In [4]:
from PipelineTS.pipeline import PipelineTS

In [5]:
from sklearn.metrics import mean_absolute_error
pipeline = PipelineTS(
    time_col='date', 
    target_col='type_a', 
    lags=30, 
    random_state=42, 
    metric=mean_absolute_error, 
    metric_less_is_better=True
)

In [6]:
pipeline.fit_and_eval(data, valid_data)

16:24:27 2023-10-07 - PipelineTS - fitting and evaluating prophet...
16:24:28 2023-10-07 - PipelineTS - fitting and evaluating auto_arima...
16:26:45 2023-10-07 - PipelineTS - fitting and evaluating catboost...
16:28:43 2023-10-07 - PipelineTS - fitting and evaluating lightgbm...
16:29:00 2023-10-07 - PipelineTS - fitting and evaluating xgboost...
16:29:32 2023-10-07 - PipelineTS - fitting and evaluating wide_gbrt...
16:30:50 2023-10-07 - PipelineTS - fitting and evaluating d_linear...
16:31:41 2023-10-07 - PipelineTS - fitting and evaluating n_linear...
16:32:23 2023-10-07 - PipelineTS - fitting and evaluating n_beats...
16:34:25 2023-10-07 - PipelineTS - fitting and evaluating n_hits...
16:37:15 2023-10-07 - PipelineTS - fitting and evaluating tcn...
16:41:57 2023-10-07 - PipelineTS - fitting and evaluating tft...


Leaderboard,model,train_cost(s),eval_cost(s),metric
0,wide_gbrt,78.431731,0.004069,405.218992
1,n_hits,169.980026,0.009572,442.879508
2,auto_arima,137.041913,0.005761,451.258799
3,n_beats,122.503421,0.008672,557.222298
4,n_linear,42.427771,0.007459,588.746517
5,d_linear,50.357118,0.008111,622.452620
6,tcn,281.105936,0.008173,674.022445
7,xgboost,31.722031,0.011376,1117.017790
8,lightgbm,16.630206,0.009281,1188.224829
9,catboost,118.625482,0.033417,1199.051870
